<a href="https://colab.research.google.com/github/JeffRubens/GitHub/blob/main/C%C3%B3pia_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini_Aula_05_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)
# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. Para saber quando será a próxima edição, o ideal é:

*   **Acompanhar as redes sociais e o site da Alura:** Eles costumam anunciar novos cursos e imersões por lá.
*   **Verificar a página de cursos de IA da Alura:** Veja se a imersão já está listada com uma data prevista.
*   **Entrar em contato com a Alura:** Você pode perguntar diretamente para a equipe deles sobre a previsão da próxima Imersão.

In [7]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
    config={'tools': [{'google_search': {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias **12 e 16 de maio de 2025.** As inscrições estão abertas e podem ser feitas no site da Alura. O curso é gratuito e online, e ao final você recebe um certificado Alura + Google.


In [8]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima imersão IA Alura Google Gemini', 'Alura Imersão IA Google Gemini datas']
Páginas utilizadas na resposta: eucapacito.com.br, alura.com.br, educacao.mg.gov.br



In [9]:
# Instalar Framework ADK de agentes do Google ################################################
#!pip install google-generative-ai-agent-framework
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [10]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [11]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [12]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [20]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do Google (google_search)
        para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
        Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
        Se um tema tiver poucas notícias ou reações entusiasmadas, é possível que ele não seja tão relevante assim
        e pode ser substituído por outro que tenha mais.
        Esses lançamentos relevantes devem ser atuais, de no máximo um mês antes a data de hoje.
        """,
        description='Agente que busca informação no Google',
        tools=[google_search]
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [17]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lançamentos mais
        recentes e relevantes buscador, você deve:
        Usar a ferramenta de busca do google (google_search) para criar um plano sobre quais são os pontos mais
        relevantes que pederíamos abordar em um post sobre cada um deles. Você também pode usar o (google_search)
        para encontrar mais informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e e retornar esse
        tema, seus pontos mais relevantes e um plano com os assuntos a serem abordados no post que será criado posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [18]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [19]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-pro-preview-03-25",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [22]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Por favor, digite um tópico válido.")
else:
  print("Maravilha! Vamos criar o post sobre novidades em {topico}")

  lancamentos_buscados = agente_buscador(topico,data_de_hoje)
  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(lancamentos_buscados))
  print("--------------------------------------------------------------------")

  plano_de_post = agente_planejador(topico,lancamentos_buscados)
  print("\n--- Resultado do Agente 2 (Planerjador) ---\n")
  display(to_markdown(plano_de_post))
  print("--------------------------------------------------------------------")

  rascunho_de_post = agente_redator(topico,plano_de_post)
  print("\n--- Resultado do Agente 3 (Redator) ---\n")
  display(to_markdown(rascunho_de_post))
  print("--------------------------------------------------------------------")

  post_final = agente_revisor(topico,rascunho_de_post)
  print("\n--- Resultado do Agente 4 (Revisor) ---\n")
  display(to_markdown(post_final))
  print("--------------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: Agentes de IA
Maravilha! Vamos criar o post sobre novidades em {topico}

--- Resultado do Agente 1 (Buscador) ---



> Para fornecer as notícias mais recentes e relevantes sobre lançamentos de agentes de IA, vou realizar algumas pesquisas no Google para identificar os lançamentos mais comentados e significativos do último mês.
> 
> 
> Com base nas pesquisas realizadas, aqui estão alguns dos lançamentos e avanços mais relevantes em agentes de IA, focando no período de aproximadamente um mês anterior à data de hoje (19 de maio de 2025):
> 
> 1.  **OpenAI Codex:** A OpenAI lançou o Codex, um agente de IA projetado para auxiliar no desenvolvimento de software. Ele é capaz de escrever recursos de software, corrigir bugs e executar testes. O Codex está sendo lançado como uma "prévia de pesquisa" para usuários pagos do ChatGPT Pro, Enterprise e Team. (Fonte: Bloomberg Linea)
> 
> 2.  **IBM watsonx Orchestrate e Agent Catalog:** A IBM anunciou atualizações no watsonx Orchestrate, oferecendo soluções para integrar, personalizar e implantar agentes de IA. Eles também lançaram o Agent Catalog, que simplifica o acesso a agentes pré-construídos para áreas como RH, vendas e compras, integrando-se com várias aplicações empresariais. (Fonte: IT Forum, Novidades em IA impulsionam a integração de agentes e transformam o futuro tecnológico)
> 
> 3.  **Avanços em Agentes Autônomos:** Empresas como Amazon (em parceria com a Anthropic), Google (com o Project Astra) e Apple estão liderando o desenvolvimento de agentes de IA autônomos capazes de realizar tarefas complexas de forma independente, acessar sistemas internos e dados proprietários. (Fonte: Exame)
> 
> 4.  **Red Hat AI:** A Red Hat tem investido no Red Hat AI, um portfólio de produtos e serviços projetados para acelerar o desenvolvimento e a implantação de soluções de inteligência artificial na nuvem híbrida. Isso inclui o Red Hat Enterprise Linux® AI e o Red Hat OpenShift® AI, que oferecem uma plataforma para o ajuste fino de modelos de linguagem e a orquestração de aplicativos, modelos e agentes. (Fonte: Bloomberg Linea)
> 
> 5.  **Frameworks Open Source para Agentes de IA:** Há um crescente interesse e desenvolvimento de frameworks open source para a criação de agentes de IA, como o Open AI Agents SDK e o MetaGPT. Essas ferramentas visam facilitar a criação de sistemas multi-agente sofisticados e fluxos de trabalho autônomos. (Fonte: ChatGPT Brasil, As 5 principais estruturas gratuitas de agentes de IA - Botpress)
> 
> Esses lançamentos refletem a tendência de agentes de IA se tornarem mais integrados em processos empresariais, auxiliando em tarefas complexas e aumentando a automação. A pesquisa indica que 2025 é um ano chave para a adoção e implementação estratégica de agentes de IA em várias indústrias.
> 


--------------------------------------------------------------------
